In [10]:
import pandas as pd

# Cargar archivo de inferencia ventas_2025_inferencia.csv en un DataFrame
from pathlib import Path
inferencia_path = Path('..') / 'data' / 'raw' / 'inferencia' / 'ventas_2025_inferencia.csv'
inferencia_df = pd.read_csv(inferencia_path)
print('Shape:', inferencia_df.shape)
inferencia_df.head()

Shape: (888, 13)


,fecha,producto_id,nombre,categoria,subcategoria,precio_base,es_estrella,unidades_vendidas,precio_venta,ingresos,Amazon,Decathlon,Deporvillage
0,2025-10-25,PROD_001,Nike Air Zoom Pegasus 40,Running,Zapatillas Running,115,True,26.0,113.13,2941.38,89.51,113.43,104.78
1,2025-10-25,PROD_002,Adidas Ultraboost 23,Running,Zapatillas Running,135,True,27.0,141.89,3831.03,128.73,112.91,122.88
2,2025-10-25,PROD_003,Asics Gel Nimbus 25,Running,Zapatillas Running,85,False,5.0,85.79,428.95,84.28,74.51,85.57
3,2025-10-25,PROD_004,New Balance Fresh Foam X 1080v12,Running,Zapatillas Running,75,False,3.0,76.19,228.57,75.54,70.32,71.13
4,2025-10-25,PROD_005,Nike Dri-FIT Miler,Running,Ropa Running,35,False,3.0,35.48,106.44,33.84,31.32,34.41


In [11]:
# --- TRANSFORMACIÓN COMPLETA DE inferencia_df PARA INFERENCIA ---

import pandas as pd
import numpy as np
import holidays

# 1. Procesamiento de fechas
inferencia_df['fecha'] = pd.to_datetime(inferencia_df['fecha'])
inferencia_df['año'] = inferencia_df['fecha'].dt.year
inferencia_df['mes'] = inferencia_df['fecha'].dt.month
inferencia_df['dia_mes'] = inferencia_df['fecha'].dt.day
inferencia_df['dia_semana'] = inferencia_df['fecha'].dt.day_name()
inferencia_df['es_fin_semana'] = inferencia_df['dia_semana'].isin(['Saturday', 'Sunday']).astype(int)
festivos_es = holidays.country_holidays('ES', years=inferencia_df['fecha'].dt.year.unique())
inferencia_df['es_festivo'] = inferencia_df['fecha'].isin(festivos_es).astype(int)

# 2. Variables de ventas: lags y media móvil
inferencia_df = inferencia_df.sort_values(['producto_id', 'fecha'])
for lag in range(1, 8):
    inferencia_df[f'unidades_vendidas_lag{lag}'] = inferencia_df.groupby('producto_id')['unidades_vendidas'].shift(lag)
inferencia_df['unidades_vendidas_ma7'] = inferencia_df.groupby('producto_id')['unidades_vendidas'].shift(1).rolling(window=7).mean()

# 3. Variables de precio
inferencia_df['descuento_porcentaje'] = ((inferencia_df['precio_venta'] - inferencia_df['precio_base']) / inferencia_df['precio_base']) * 100

# 4. Variables de competencia
competidores = ['Amazon', 'Decathlon', 'Deporvillage']
if all(col in inferencia_df.columns for col in competidores):
    inferencia_df['precio_competencia'] = inferencia_df[competidores].mean(axis=1)
    inferencia_df['ratio_precio'] = inferencia_df['precio_venta'] / inferencia_df['precio_competencia']
    inferencia_df = inferencia_df.drop(columns=competidores)
else:
    # Si no existen, usa el promedio histórico por producto (requiere df_historico cargado)
    for producto_id in inferencia_df['producto_id'].unique():
        precio_comp_hist = df_historico[df_historico['producto_id'] == producto_id]['precio_competencia'].mean()
        inferencia_df.loc[inferencia_df['producto_id'] == producto_id, 'precio_competencia'] = precio_comp_hist
    inferencia_df['ratio_precio'] = inferencia_df['precio_venta'] / inferencia_df['precio_competencia']

# 5. Codificación categórica (One Hot Encoding)
for col in ['nombre', 'categoria', 'subcategoria']:
    if col in inferencia_df.columns:
        inferencia_df[f'{col}_h'] = inferencia_df[col]
inferencia_df = pd.get_dummies(inferencia_df, columns=['nombre_h', 'categoria_h', 'subcategoria_h'], drop_first=False)

# 6. Eliminar registros de octubre y dejar solo noviembre
inferencia_df = inferencia_df[inferencia_df['mes'] == 11].copy()

# 7. Guardar el DataFrame transformado
ruta_final = './data/processed/inferencia_df_transformado.csv'
inferencia_df.to_csv(ruta_final, index=False)
print(f"\n✅ DataFrame transformado guardado en: {ruta_final}")


✅ DataFrame transformado guardado en: ./data/processed/inferencia_df_transformado.csv


C:\Users\Usuario\AppData\Local\Temp\ipykernel_9620\3316482759.py:15: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  inferencia_df['es_festivo'] = inferencia_df['fecha'].isin(festivos_es).astype(int)


In [12]:
inferencia_df

,fecha,producto_id,nombre,categoria,subcategoria,precio_base,es_estrella,unidades_vendidas,precio_venta,ingresos,...,subcategoria_h_Esterilla Yoga,subcategoria_h_Mancuernas Ajustables,subcategoria_h_Mochila Trekking,subcategoria_h_Pesa Rusa,subcategoria_h_Pesas Casa,subcategoria_h_Rodillera Yoga,subcategoria_h_Ropa Montaña,subcategoria_h_Ropa Running,subcategoria_h_Zapatillas Running,subcategoria_h_Zapatillas Trail
168,2025-11-01,PROD_001,Nike Air Zoom Pegasus 40,Running,Zapatillas Running,115,True,NaN,115.00,NaN,...,False,False,False,False,False,False,False,False,True,False
192,2025-11-02,PROD_001,Nike Air Zoom Pegasus 40,Running,Zapatillas Running,115,True,NaN,115.00,NaN,...,False,False,False,False,False,False,False,False,True,False
216,2025-11-03,PROD_001,Nike Air Zoom Pegasus 40,Running,Zapatillas Running,115,True,NaN,115.00,NaN,...,False,False,False,False,False,False,False,False,True,False
240,2025-11-04,PROD_001,Nike Air Zoom Pegasus 40,Running,Zapatillas Running,115,True,NaN,115.00,NaN,...,False,False,False,False,False,False,False,False,True,False
264,2025-11-05,PROD_001,Nike Air Zoom Pegasus 40,Running,Zapatillas Running,115,True,NaN,115.00,NaN,...,False,False,False,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
791,2025-11-26,PROD_024,Lotuscrafts Yoga Bolster,Wellness,Cojín Yoga,50,False,NaN,50.85,NaN,...,False,False,False,False,False,False,False,False,False,False
815,2025-11-27,PROD_024,Lotuscrafts Yoga Bolster,Wellness,Cojín Yoga,50,False,NaN,49.82,NaN,...,False,False,False,False,False,False,False,False,False,False
839,2025-11-28,PROD_024,Lotuscrafts Yoga Bolster,Wellness,Cojín Yoga,50,False,NaN,42.50,NaN,...,False,False,False,False,False,False,False,False,False,False
863,2025-11-29,PROD_024,Lotuscrafts Yoga Bolster,Wellness,Cojín Yoga,50,False,NaN,50.37,NaN,...,False,False,False,False,False,False,False,False,False,False


In [14]:
inferencia_df.shape

(720, 71)

In [18]:
inferencia_df.fecha.unique()

<DatetimeArray>
['2025-11-01 00:00:00', '2025-11-02 00:00:00', '2025-11-03 00:00:00',
 '2025-11-04 00:00:00', '2025-11-05 00:00:00', '2025-11-06 00:00:00',
 '2025-11-07 00:00:00', '2025-11-08 00:00:00', '2025-11-09 00:00:00',
 '2025-11-10 00:00:00', '2025-11-11 00:00:00', '2025-11-12 00:00:00',
 '2025-11-13 00:00:00', '2025-11-14 00:00:00', '2025-11-15 00:00:00',
 '2025-11-16 00:00:00', '2025-11-17 00:00:00', '2025-11-18 00:00:00',
 '2025-11-19 00:00:00', '2025-11-20 00:00:00', '2025-11-21 00:00:00',
 '2025-11-22 00:00:00', '2025-11-23 00:00:00', '2025-11-24 00:00:00',
 '2025-11-25 00:00:00', '2025-11-26 00:00:00', '2025-11-27 00:00:00',
 '2025-11-28 00:00:00', '2025-11-29 00:00:00', '2025-11-30 00:00:00']
Length: 30, dtype: datetime64[ns]

In [ ]:
-------------------------------------------------------------------------------------------------------

In [3]:
# Importar librerías principales (igual que 1bENTRENMIENTO.IPYNB)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import sklearn
import streamlit as st
import holidays

# Forecasting - Importación de librerías y carga de datos
Este notebook importa las mismas librerías que el notebook de entrenamiento y carga el archivo de inferencia para análisis.

## Forecasting de ventas 2025
Este notebook importa las librerías necesarias y carga los datos de ventas para inferencia.

In [4]:
# Imports principales para el forecasting
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import streamlit as st
import holidays

## Cargar archivo de ventas para inferencia
Se carga el archivo 'data/raw/inferencia/ventas_2025_inferencia.csv' en un DataFrame llamado inferencia_df y se muestran las primeras filas para verificar la carga.

In [5]:
import pandas as pd

# 1. Definimos la ruta exacta usando una "r" antes de las comillas para evitar errores de sintaxis en Windows
inferencia_path = r"C:\Users\Usuario\Desktop\Data Science Proyecto\data\raw\inferencia\ventas_2025_inferencia.csv"

# 2. Cargamos el archivo
try:
    inferencia_df = pd.read_csv(inferencia_path)
    print("¡Archivo cargado con éxito!")
    
    # 3. Mostramos las primeras filas
    display(inferencia_df.head())
except FileNotFoundError:
    print("Error: No se encontró el archivo. Revisa que el nombre 'ventas_2025_inferencia.csv' sea correcto.")
except Exception as e:
    print(f"Ocurrió un error inesperado: {e}")

¡Archivo cargado con éxito!


,fecha,producto_id,nombre,categoria,subcategoria,precio_base,es_estrella,unidades_vendidas,precio_venta,ingresos,Amazon,Decathlon,Deporvillage
0,2025-10-25,PROD_001,Nike Air Zoom Pegasus 40,Running,Zapatillas Running,115,True,26.0,113.13,2941.38,89.51,113.43,104.78
1,2025-10-25,PROD_002,Adidas Ultraboost 23,Running,Zapatillas Running,135,True,27.0,141.89,3831.03,128.73,112.91,122.88
2,2025-10-25,PROD_003,Asics Gel Nimbus 25,Running,Zapatillas Running,85,False,5.0,85.79,428.95,84.28,74.51,85.57
3,2025-10-25,PROD_004,New Balance Fresh Foam X 1080v12,Running,Zapatillas Running,75,False,3.0,76.19,228.57,75.54,70.32,71.13
4,2025-10-25,PROD_005,Nike Dri-FIT Miler,Running,Ropa Running,35,False,3.0,35.48,106.44,33.84,31.32,34.41


In [6]:
import pandas as pd
import numpy as np
import holidays
import os

# 1. Convertir fecha
inferencia_df['fecha'] = pd.to_datetime(inferencia_df['fecha'])

# 2. Variables temporales básicas
inferencia_df['año'] = inferencia_df['fecha'].dt.year
inferencia_df['mes'] = inferencia_df['fecha'].dt.month
inferencia_df['dia_mes'] = inferencia_df['fecha'].dt.day
inferencia_df['dia_semana'] = inferencia_df['fecha'].dt.day_name()
inferencia_df['es_fin_semana'] = inferencia_df['dia_semana'].isin(['Saturday', 'Sunday']).astype(int)

# 3. Festivos
festivos_es = holidays.country_holidays('ES', years=inferencia_df['fecha'].dt.year.unique())
inferencia_df['es_festivo'] = inferencia_df['fecha'].isin(festivos_es).astype(int)

# 13. (MOVIDO ARRIBA) Gestión de Precios de Competencia
# Calculamos primero para tener las referencias de precio antes de los lags
columnas_comp = ['Amazon', 'Decathlon', 'Deporvillage']
if set(columnas_comp).issubset(inferencia_df.columns):
    # El precio de la competencia es el mínimo encontrado en esas tiendas
    inferencia_df['precio_competencia'] = inferencia_df[columnas_comp].min(axis=1)
    # Ratio entre nuestro precio de venta y la competencia
    inferencia_df['ratio_precio'] = inferencia_df['precio_venta'] / inferencia_df['precio_competencia']
    # IMPORTANTE: No hacemos .drop todavía si necesitas esos datos para las unidades

# 10. Lags y media móvil (Basado en 'unidades_vendidas')
# Si 'unidades_vendidas' no viene calculada, asegúrate de que exista la columna
if 'unidades_vendidas' not in inferencia_df.columns:
    print("Advertencia: 'unidades_vendidas' no encontrada. Se creará con ceros para el cálculo.")
    inferencia_df['unidades_vendidas'] = 0 

lags = range(1, 8)
for lag in lags:
    inferencia_df[f'unidades_vendidas_lag{lag}'] = inferencia_df.groupby('año')['unidades_vendidas'].shift(lag)

# Media móvil de 7 días
for year in inferencia_df['año'].unique():
    mask = inferencia_df['año'] == year
    inferencia_df.loc[mask, 'unidades_vendidas_ma7'] = (
        inferencia_df.loc[mask, 'unidades_vendidas'].shift(1).rolling(window=7).mean()
    )

# 11. Limpieza de Nulos (necesario tras crear lags)
inferencia_df = inferencia_df.dropna(subset=[f'unidades_vendidas_lag{l}' for l in lags] + ['unidades_vendidas_ma7'])

# 12. Variable de descuento (Usando 'precio_base' detectado en entrenamiento)
# Cambié 'precio_original' por 'precio_base' para evitar el KeyError anterior
inferencia_df['descuento_porcentaje'] = (
    (inferencia_df['precio_base'] - inferencia_df['precio_venta']) / inferencia_df['precio_base']
) * 100

# 14. Filtrado final: Solo Noviembre y limpieza de columnas sobrantes
inferencia_df = inferencia_df[inferencia_df['fecha'].dt.month == 11]

# Ahora sí podemos limpiar las columnas de competencia originales si ya no las usas
columnas_a_borrar = [c for c in columnas_comp if c in inferencia_df.columns]
if columnas_a_borrar:
    inferencia_df = inferencia_df.drop(columns=columnas_a_borrar)

# Guardar
output_path = "../data/processed/inferencia_df_transformado.csv"
os.makedirs("../data/processed", exist_ok=True)
inferencia_df.to_csv(output_path, index=False)

print(f"OK. Archivo guardado con éxito en {output_path}")
display(inferencia_df.head())

OK. Archivo guardado con éxito en ../data/processed/inferencia_df_transformado.csv


C:\Users\Usuario\AppData\Local\Temp\ipykernel_9620\2095990916.py:18: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  inferencia_df['es_festivo'] = inferencia_df['fecha'].isin(festivos_es).astype(int)


,fecha,producto_id,nombre,categoria,subcategoria,precio_base,es_estrella,unidades_vendidas,precio_venta,ingresos,...,ratio_precio,unidades_vendidas_lag1,unidades_vendidas_lag2,unidades_vendidas_lag3,unidades_vendidas_lag4,unidades_vendidas_lag5,unidades_vendidas_lag6,unidades_vendidas_lag7,unidades_vendidas_ma7,descuento_porcentaje
168,2025-11-01,PROD_001,Nike Air Zoom Pegasus 40,Running,Zapatillas Running,115,True,NaN,115.0,NaN,...,1.406212,1.0,6.0,3.0,6.0,3.0,2.0,2.0,3.285714,0.0


In [7]:
import pandas as pd
import numpy as np

# 1. Carga df procesado con la ruta y nombre de archivo correctos
# Usamos ./data/processed/DF.csv basado en tu estructura de carpetas
df_historico = pd.read_csv('./data/processed/DF.csv')
df_historico['fecha'] = pd.to_datetime(df_historico['fecha'])

# Aseguramos que inferencia_df tenga formato datetime y la columna 'mes'
inferencia_df['fecha'] = pd.to_datetime(inferencia_df['fecha'])
inferencia_df['mes'] = inferencia_df['fecha'].dt.month

# Configuración de lags
lags = range(1, 8)

# Ordenar por producto y fecha para que los lags tengan sentido cronológico
inferencia_df = inferencia_df.sort_values(['producto_id', 'fecha'])

# Bucle para procesar cada producto
for producto_id in inferencia_df['producto_id'].unique():
    # Obtener últimos 7 registros del histórico para este producto
    df_prod_hist = df_historico[df_historico['producto_id'] == producto_id].sort_values('fecha').tail(7)
    valores_hist = df_prod_hist['unidades_vendidas'].values

    # Obtener registros de inferencia para este producto
    mask_prod = inferencia_df['producto_id'] == producto_id
    indices_prod = inferencia_df[mask_prod].index

    # Inicializar lags y media móvil
    for i, idx in enumerate(indices_prod):
        # Lags: Si no hay suficiente historia en inferencia, busca en valores_hist
        for lag in lags:
            col_name = f'unidades_vendidas_lag{lag}'
            if i >= lag:
                inferencia_df.loc[idx, col_name] = inferencia_df.loc[indices_prod[i-lag], 'unidades_vendidas']
            else:
                hist_idx = len(valores_hist) - lag + i
                if hist_idx >= 0:
                    inferencia_df.loc[idx, col_name] = valores_hist[hist_idx]
                else:
                    inferencia_df.loc[idx, col_name] = np.nan

        # Media móvil de 7 días (MA7)
        valores_ma = []
        for j in range(1, 8): # Miramos los 7 días anteriores
            if i - j >= 0:
                valores_ma.append(inferencia_df.loc[indices_prod[i-j], 'unidades_vendidas'])
            else:
                hist_idx = len(valores_hist) + (i - j)
                if hist_idx >= 0:
                    valores_ma.append(valores_hist[hist_idx])
        
        if len(valores_ma) > 0:
            inferencia_df.loc[idx, 'unidades_vendidas_ma7'] = np.mean(valores_ma)
        else:
            inferencia_df.loc[idx, 'unidades_vendidas_ma7'] = np.nan

# --- IMPRESIONES DE CONTROL ---

print("Registros después de crear lags: ", len(inferencia_df))

# Columnas de lags para el reporte
cols_lags = [f'unidades_vendidas_lag{lag}' for lag in lags]
cols_reporte = cols_lags + ['unidades_vendidas_ma7']

print("Registros con nulos en lags:")
print(inferencia_df[cols_reporte].isnull().sum())

print("\nN primeros registros con lags (Vista previa):")
# 'mes' y 'fecha' ahora están garantizados
print(inferencia_df[cols_reporte + ['fecha', 'mes']].head(10))


Registros después de crear lags:  1
Registros con nulos en lags:
unidades_vendidas_lag1    0
unidades_vendidas_lag2    0
unidades_vendidas_lag3    0
unidades_vendidas_lag4    0
unidades_vendidas_lag5    0
unidades_vendidas_lag6    0
unidades_vendidas_lag7    0
unidades_vendidas_ma7     0
dtype: int64

N primeros registros con lags (Vista previa):
     unidades_vendidas_lag1  unidades_vendidas_lag2  unidades_vendidas_lag3  \
168                    80.0                    66.0                    15.0   

     unidades_vendidas_lag4  unidades_vendidas_lag5  unidades_vendidas_lag6  \
168                    13.0                    18.0                    12.0   

     unidades_vendidas_lag7  unidades_vendidas_ma7      fecha  mes  
168                    16.0              31.428571 2025-11-01   11  


In [8]:
# 1. Convertir la columna 'fecha' a tipo datetime (como en tu imagen)
inferencia_df['fecha'] = pd.to_datetime(inferencia_df['fecha'])

# 2. Verificar la conversión (opcional, según tu imagen)
print("Tipo de fecha en inferencia_df:", inferencia_df['fecha'].dtype)

# 3. ADAPTACIÓN: Crear la columna 'mes' necesaria para el resto de tu código
# Esto evita el error KeyError: "['mes'] not in index"
inferencia_df['mes'] = inferencia_df['fecha'].dt.month

# 4. Verificar que 'mes' se creó correctamente
print("Columnas actuales:", inferencia_df.columns.tolist())

Tipo de fecha en inferencia_df: datetime64[ns]
Columnas actuales: ['fecha', 'producto_id', 'nombre', 'categoria', 'subcategoria', 'precio_base', 'es_estrella', 'unidades_vendidas', 'precio_venta', 'ingresos', 'año', 'mes', 'dia_mes', 'dia_semana', 'es_fin_semana', 'es_festivo', 'precio_competencia', 'ratio_precio', 'unidades_vendidas_lag1', 'unidades_vendidas_lag2', 'unidades_vendidas_lag3', 'unidades_vendidas_lag4', 'unidades_vendidas_lag5', 'unidades_vendidas_lag6', 'unidades_vendidas_lag7', 'unidades_vendidas_ma7', 'descuento_porcentaje']


In [9]:
import pandas as pd

# --- ADAPTACIÓN IMAGEN 1: Conversión de fecha y creación de 'mes' ---
# Convertir la columna 'fecha' a tipo datetime
inferencia_df['fecha'] = pd.to_datetime(inferencia_df['fecha'])

# IMPORTANTE: Creamos la columna 'mes' para evitar el KeyError detectado anteriormente
inferencia_df['mes'] = inferencia_df['fecha'].dt.month

# Verificar la conversión
print("Tipo de fecha en inferencia_df:", inferencia_df['fecha'].dtype)


# --- ADAPTACIÓN IMAGEN 2: Variable de descuento ---
# Fórmula: ((precio_venta - precio_base) / precio_base) * 100
inferencia_df['descuento_porcentaje'] = ((inferencia_df['precio_venta'] - inferencia_df['precio_base']) / inferencia_df['precio_base']) * 100

# Mostrar las primeras filas para verificar (incluyendo 'mes' para confirmar que existe)
print("\nVerificación de nuevas columnas (mes y descuento):")
columnas_verificacion = ['fecha', 'mes', 'producto_id', 'precio_base', 'precio_venta', 'descuento_porcentaje']
print(inferencia_df[columnas_verificacion].head())

Tipo de fecha en inferencia_df: datetime64[ns]

Verificación de nuevas columnas (mes y descuento):
         fecha  mes producto_id  precio_base  precio_venta  \
168 2025-11-01   11    PROD_001          115         115.0   

     descuento_porcentaje  
168                   0.0  


In [ ]:
import pandas as pd

# --- 1. PROCESAMIENTO DE FECHAS (Basado en imagen 1) ---
# Convertir la columna 'fecha' a tipo datetime
inferencia_df['fecha'] = pd.to_datetime(inferencia_df['fecha'])

# ADAPTACIÓN NECESARIA: Creamos la columna 'mes' para evitar KeyErrors futuros
inferencia_df['mes'] = inferencia_df['fecha'].dt.month

print("Tipo de fecha en inferencia_df:", inferencia_df['fecha'].dtype)


# --- 2. CÁLCULO DE DESCUENTO (Basado en imagen 2) ---
# Fórmula: ((precio_venta - precio_base) / precio_base) * 100
inferencia_df['descuento_porcentaje'] = ((inferencia_df['precio_venta'] - inferencia_df['precio_base']) / inferencia_df['precio_base']) * 100


# --- 3. ANÁLISIS DE COMPETENCIA (Basado en imagen 3) ---
# Verificamos si existen las columnas de competidores específicos
competidores = ['Amazon', 'Decathlon', 'Deporvillage']

if all(col in inferencia_df.columns for col in competidores):
    # Caso A: Si las columnas existen, calculamos el promedio directo
    inferencia_df['precio_competencia'] = inferencia_df[competidores].mean(axis=1)
    # Variable ratio_precio: nuestro precio vs competencia
    inferencia_df['ratio_precio'] = inferencia_df['precio_venta'] / inferencia_df['precio_competencia']
    # Limpiamos las columnas originales de competidores
    inferencia_df = inferencia_df.drop(columns=competidores)
else:
    # Caso B: Si no existen, usamos el promedio histórico de df_historico por producto
    for producto_id in inferencia_df['producto_id'].unique():
        precio_comp_hist = df_historico[df_historico['producto_id'] == producto_id]['precio_competencia'].mean()
        inferencia_df.loc[inferencia_df['producto_id'] == producto_id, 'precio_competencia'] = precio_comp_hist
    
    # Calculamos el ratio con los valores históricos obtenidos
    inferencia_df['ratio_precio'] = inferencia_df['precio_venta'] / inferencia_df['precio_competencia']


# --- VERIFICACIÓN FINAL ---
print("\nPrimeras filas del DataFrame procesado:")
columnas_vista = ['fecha', 'mes', 'producto_id', 'precio_venta', 'precio_competencia', 'ratio_precio', 'descuento_porcentaje']
print(inferencia_df[columnas_vista].head())

Tipo de fecha en inferencia_df: datetime64[ns]

Primeras filas del DataFrame procesado:
        fecha  mes producto_id  precio_venta  precio_competencia  \
0  2025-10-25   10    PROD_001        113.13          100.974426   
24 2025-10-26   10    PROD_001        105.75          100.974426   
48 2025-10-27   10    PROD_001        114.95          100.974426   
72 2025-10-28   10    PROD_001        117.31          100.974426   
96 2025-10-29   10    PROD_001        108.10          100.974426   

    ratio_precio  descuento_porcentaje  
0       1.120383             -1.626087  
24      1.047295             -8.043478  
48      1.138407             -0.043478  
72      1.161779              2.008696  
96      1.070568             -6.000000  


In [ ]:
import pandas as pd

# --- 1. PROCESAMIENTO DE FECHAS (Imagen 1) ---
# Convertir la columna 'fecha' a tipo datetime
inferencia_df['fecha'] = pd.to_datetime(inferencia_df['fecha'])
# Extraemos el 'mes' para evitar errores de referencia y facilitar el filtrado final
inferencia_df['mes'] = inferencia_df['fecha'].dt.month
print(f"Tipo de fecha en inferencia_df: {inferencia_df['fecha'].dtype}")

# --- 2. VARIABLE DE DESCUENTO (Imagen 2) ---
# Fórmula: ((precio_venta - precio_base) / precio_base) * 100
inferencia_df['descuento_porcentaje'] = ((inferencia_df['precio_venta'] - inferencia_df['precio_base']) / inferencia_df['precio_base']) * 100
print("\nVerificación de descuento:")
print(inferencia_df[['fecha', 'producto_id', 'precio_base', 'precio_venta', 'descuento_porcentaje']].head())

# --- 3. ANÁLISIS DE COMPETENCIA (Imágenes 3 y 4) ---
competidores = ['Amazon', 'Decathlon', 'Deporvillage']

# Si las columnas existen, calculamos el promedio directamente
if all(col in inferencia_df.columns for col in competidores):
    inferencia_df['precio_competencia'] = inferencia_df[competidores].mean(axis=1)
    inferencia_df['ratio_precio'] = inferencia_df['precio_venta'] / inferencia_df['precio_competencia']
    # Limpiamos las columnas originales para dejar solo el promedio
    inferencia_df = inferencia_df.drop(columns=competidores)
else:
    # Si no existen, usamos el promedio histórico por producto de df_historico
    for producto_id in inferencia_df['producto_id'].unique():
        precio_comp_hist = df_historico[df_historico['producto_id'] == producto_id]['precio_competencia'].mean()
        inferencia_df.loc[inferencia_df['producto_id'] == producto_id, 'precio_competencia'] = precio_comp_hist
    
    # Calculamos el ratio con los valores históricos obtenidos
    inferencia_df['ratio_precio'] = inferencia_df['precio_venta'] / inferencia_df['precio_competencia']

# --- 4. FILTRADO Y GUARDADO FINAL (Imágenes 5 y 6) ---
print("\n--- Estadísticas antes del filtrado ---")
print(f"Registros totales antes del filtrado: {len(inferencia_df)}")
print(f"Registros de octubre: {len(inferencia_df[inferencia_df['mes'] == 10])}")
print(f"Registros de noviembre: {len(inferencia_df[inferencia_df['mes'] == 11])}")

# Eliminar todos los registros de octubre y mantener solo noviembre
inferencia_df = inferencia_df[inferencia_df['mes'] == 11].copy()

print(f"\nRegistros después de filtrar solo noviembre: {len(inferencia_df)}")
print(f"Rango de fechas: {inferencia_df['fecha'].min()} a {inferencia_df['fecha'].max()}")
print(f"Productos únicos: {inferencia_df['producto_id'].nunique()}")

# Verificar si hay nulos en las variables de lags para noviembre
cols_lag_ma = [f'unidades_vendidas_lag{lag}' for lag in range(1, 8)] + ['unidades_vendidas_ma7']
nulos_noviembre = inferencia_df[cols_lag_ma].isnull().sum()

if nulos_noviembre.sum() > 0:
    print(f"\n⚠️ Atención: Hay {nulos_noviembre.sum()} valores nulos en variables de lag en noviembre:")
    print(nulos_noviembre[nulos_noviembre > 0])
else:
    print("\n✅ No hay valores nulos en las variables de lag para noviembre")

Tipo de fecha en inferencia_df: datetime64[ns]

Verificación de descuento:
        fecha producto_id  precio_base  precio_venta  descuento_porcentaje
0  2025-10-25    PROD_001          115        113.13             -1.626087
24 2025-10-26    PROD_001          115        105.75             -8.043478
48 2025-10-27    PROD_001          115        114.95             -0.043478
72 2025-10-28    PROD_001          115        117.31              2.008696
96 2025-10-29    PROD_001          115        108.10             -6.000000

--- Estadísticas antes del filtrado ---
Registros totales antes del filtrado: 888
Registros de octubre: 168
Registros de noviembre: 720

Registros después de filtrar solo noviembre: 720
Rango de fechas: 2025-11-01 00:00:00 a 2025-11-30 00:00:00
Productos únicos: 24

⚠️ Atención: Hay 5064 valores nulos en variables de lag en noviembre:
unidades_vendidas_lag1    696
unidades_vendidas_lag2    672
unidades_vendidas_lag3    648
unidades_vendidas_lag4    624
unidades_vendidas

In [ ]:
import pandas as pd

# --- 1. PROCESAMIENTO DE FECHAS (Imagen 1) ---
# Convertir la columna 'fecha' a tipo datetime
inferencia_df['fecha'] = pd.to_datetime(inferencia_df['fecha'])
# Extraemos el 'mes' para evitar errores de referencia y facilitar el filtrado final
inferencia_df['mes'] = inferencia_df['fecha'].dt.month
print(f"Tipo de fecha en inferencia_df: {inferencia_df['fecha'].dtype}")

# --- 2. VARIABLE DE DESCUENTO (Imagen 2) ---
# Fórmula: ((precio_venta - precio_base) / precio_base) * 100
inferencia_df['descuento_porcentaje'] = ((inferencia_df['precio_venta'] - inferencia_df['precio_base']) / inferencia_df['precio_base']) * 100
print("\nVerificación de descuento:")
print(inferencia_df[['fecha', 'producto_id', 'precio_base', 'precio_venta', 'descuento_porcentaje']].head())

# --- 3. ANÁLISIS DE COMPETENCIA (Imágenes 3 y 4) ---
competidores = ['Amazon', 'Decathlon', 'Deporvillage']

# Si las columnas existen, calculamos el promedio directamente
if all(col in inferencia_df.columns for col in competidores):
    inferencia_df['precio_competencia'] = inferencia_df[competidores].mean(axis=1)
    inferencia_df['ratio_precio'] = inferencia_df['precio_venta'] / inferencia_df['precio_competencia']
    # Limpiamos las columnas originales para dejar solo el promedio
    inferencia_df = inferencia_df.drop(columns=competidores)
else:
    # Si no existen, usamos el promedio histórico por producto de df_historico
    for producto_id in inferencia_df['producto_id'].unique():
        precio_comp_hist = df_historico[df_historico['producto_id'] == producto_id]['precio_competencia'].mean()
        inferencia_df.loc[inferencia_df['producto_id'] == producto_id, 'precio_competencia'] = precio_comp_hist
    
    # Calculamos el ratio con los valores históricos obtenidos
    inferencia_df['ratio_precio'] = inferencia_df['precio_venta'] / inferencia_df['precio_competencia']

# --- 4. FILTRADO Y GUARDADO FINAL (Imágenes 5 y 6) ---
print("\n--- Estadísticas antes del filtrado ---")
print(f"Registros totales antes del filtrado: {len(inferencia_df)}")
print(f"Registros de octubre: {len(inferencia_df[inferencia_df['mes'] == 10])}")
print(f"Registros de noviembre: {len(inferencia_df[inferencia_df['mes'] == 11])}")

# Eliminar todos los registros de octubre y mantener solo noviembre
inferencia_df = inferencia_df[inferencia_df['mes'] == 11].copy()

print(f"\nRegistros después de filtrar solo noviembre: {len(inferencia_df)}")
print(f"Rango de fechas: {inferencia_df['fecha'].min()} a {inferencia_df['fecha'].max()}")
print(f"Productos únicos: {inferencia_df['producto_id'].nunique()}")

# Verificar si hay nulos en las variables de lags para noviembre
cols_lag_ma = [f'unidades_vendidas_lag{lag}' for lag in range(1, 8)] + ['unidades_vendidas_ma7']
nulos_noviembre = inferencia_df[cols_lag_ma].isnull().sum()

if nulos_noviembre.sum() > 0:
    print(f"\n⚠️ Atención: Hay {nulos_noviembre.sum()} valores nulos en variables de lag en noviembre:")
    print(nulos_noviembre[nulos_noviembre > 0])
else:
    print("\n✅ No hay valores nulos en las variables de lag para noviembre")

Tipo de fecha en inferencia_df: datetime64[ns]

Verificación de descuento:
         fecha producto_id  precio_base  precio_venta  descuento_porcentaje
168 2025-11-01    PROD_001          115         115.0                   0.0
192 2025-11-02    PROD_001          115         115.0                   0.0
216 2025-11-03    PROD_001          115         115.0                   0.0
240 2025-11-04    PROD_001          115         115.0                   0.0
264 2025-11-05    PROD_001          115         115.0                   0.0

--- Estadísticas antes del filtrado ---
Registros totales antes del filtrado: 720
Registros de octubre: 0
Registros de noviembre: 720

Registros después de filtrar solo noviembre: 720
Rango de fechas: 2025-11-01 00:00:00 a 2025-11-30 00:00:00
Productos únicos: 24

⚠️ Atención: Hay 5064 valores nulos en variables de lag en noviembre:
unidades_vendidas_lag1    696
unidades_vendidas_lag2    672
unidades_vendidas_lag3    648
unidades_vendidas_lag4    624
unidades_vend

In [ ]:
import pandas as pd
import numpy as np

# --- 1. PROCESAMIENTO DE FECHAS Y COLUMNA 'MES' (Imagen 1) ---
# Convertir la columna 'fecha' a tipo datetime
inferencia_df['fecha'] = pd.to_datetime(inferencia_df['fecha'])
# ADAPTACIÓN: Crear la columna 'mes' necesaria para el filtrado y evitar KeyErrors
inferencia_df['mes'] = inferencia_df['fecha'].dt.month
print(f"Tipo de fecha en inferencia_df: {inferencia_df['fecha'].dtype}")

# --- 2. VARIABLE DE DESCUENTO PORCENTAJE (Imagen 2) ---
# Fórmula: ((precio_venta - precio_base) / precio_base) * 100
inferencia_df['descuento_porcentaje'] = ((inferencia_df['precio_venta'] - inferencia_df['precio_base']) / inferencia_df['precio_base']) * 100
print("\nVerificación de cálculo de descuento:")
print(inferencia_df[['fecha', 'producto_id', 'precio_base', 'precio_venta', 'descuento_porcentaje']].head())

# --- 3. ANÁLISIS DE COMPETENCIA (Imágenes 3 y 4) ---
competidores = ['Amazon', 'Decathlon', 'Deporvillage']

# Verificamos si las columnas de competencia existen en el DataFrame actual
if all(col in inferencia_df.columns for col in competidores):
    # Caso A: Calculamos el promedio de la competencia y el ratio nuestro precio vs competencia
    inferencia_df['precio_competencia'] = inferencia_df[competidores].mean(axis=1)
    inferencia_df['ratio_precio'] = inferencia_df['precio_venta'] / inferencia_df['precio_competencia']
    # Limpiamos las columnas originales de competidores
    inferencia_df = inferencia_df.drop(columns=competidores)
else:
    # Caso B: Usar el promedio histórico por producto si no hay datos actuales
    for producto_id in inferencia_df['producto_id'].unique():
        precio_comp_hist = df_historico[df_historico['producto_id'] == producto_id]['precio_competencia'].mean()
        inferencia_df.loc[inferencia_df['producto_id'] == producto_id, 'precio_competencia'] = precio_comp_hist
    
    # Calculamos el ratio con los valores históricos obtenidos
    inferencia_df['ratio_precio'] = inferencia_df['precio_venta'] / inferencia_df['precio_competencia']

# --- 4. FILTRADO Y GUARDADO FINAL (Imágenes 5, 6 y 7) ---
print("\n--- Estadísticas antes del filtrado ---")
print(f"Registros totales antes del filtrado: {len(inferencia_df)}")
print(f"Registros de octubre (mes 10): {len(inferencia_df[inferencia_df['mes'] == 10])}")
print(f"Registros de noviembre (mes 11): {len(inferencia_df[inferencia_df['mes'] == 11])}")

# Eliminar todos los registros de octubre (usados para calcular lags) y mantener solo noviembre
inferencia_df = inferencia_df[inferencia_df['mes'] == 11].copy()

print(f"\nRegistros después de filtrar solo noviembre: {len(inferencia_df)}")
print(f"Fechas en inferencia_df: {inferencia_df['fecha'].min()} a {inferencia_df['fecha'].max()}")
print(f"Productos únicos: {inferencia_df['producto_id'].nunique()}")

# Verificar si hay nulos en las variables de lags para el set final de noviembre
cols_lag_ma = [f'unidades_vendidas_lag{lag}' for lag in range(1, 8)] + ['unidades_vendidas_ma7']
nulos_noviembre = inferencia_df[cols_lag_ma].isnull().sum()

if nulos_noviembre.sum() > 0:
    print(f"\n⚠️ Atención: Hay {nulos_noviembre.sum()} valores nulos en variables de lag en noviembre:")
    print(nulos_noviembre[nulos_noviembre > 0])
else:
    print("\n✅ No hay valores nulos en las variables de lag para noviembre")

# --- 5. GUARDAR RESULTADOS (Imagen 7) ---
ruta_guardado = './data/processed/inferencia_df_transformado.csv'
inferencia_df.to_csv(ruta_guardado, index=False)
print(f"\nDataFrame transformado guardado en: {ruta_guardado}")

Tipo de fecha en inferencia_df: datetime64[ns]

Verificación de cálculo de descuento:
         fecha producto_id  precio_base  precio_venta  descuento_porcentaje
168 2025-11-01    PROD_001          115         115.0                   0.0
192 2025-11-02    PROD_001          115         115.0                   0.0
216 2025-11-03    PROD_001          115         115.0                   0.0
240 2025-11-04    PROD_001          115         115.0                   0.0
264 2025-11-05    PROD_001          115         115.0                   0.0

--- Estadísticas antes del filtrado ---
Registros totales antes del filtrado: 720
Registros de octubre (mes 10): 0
Registros de noviembre (mes 11): 720

Registros después de filtrar solo noviembre: 720
Fechas en inferencia_df: 2025-11-01 00:00:00 a 2025-11-30 00:00:00
Productos únicos: 24

⚠️ Atención: Hay 5064 valores nulos en variables de lag en noviembre:
unidades_vendidas_lag1    696
unidades_vendidas_lag2    672
unidades_vendidas_lag3    648
unidad

In [ ]:
import pandas as pd

# --- FILTRADO FINAL (Basado en tus imágenes de control) ---
print("Filtrando solo noviembre y guardando el DataFrame transformado...")

# 1. Estadísticas previas al filtrado para control
print(f"Registros totales antes del filtrado: {len(inferencia_df)}")
print(f"Registros de octubre: {len(inferencia_df[inferencia_df['mes'] == 10])}")
print(f"Registros de noviembre: {len(inferencia_df[inferencia_df['mes'] == 11])}")

# 2. Eliminar registros de octubre (usados solo para cálculos) y mantener noviembre
inferencia_df = inferencia_df[inferencia_df['mes'] == 11].copy()

# 3. Verificación de nulos en variables técnicas de noviembre
cols_lag_ma = [f'unidades_vendidas_lag{lag}' for lag in range(1, 8)] + ['unidades_vendidas_ma7']
nulos_noviembre = inferencia_df[cols_lag_ma].isnull().sum()

if nulos_noviembre.sum() > 0:
    print(f"\n⚠️ Atención: Hay {nulos_noviembre.sum()} valores nulos en variables de lag en noviembre:")
    print(nulos_noviembre[nulos_noviembre > 0])
else:
    print("\n✅ No hay valores nulos en las variables de lag para noviembre")

# --- GUARDADO ADAPTADO A TU ESTRUCTURA ---
# Usamos la ruta relativa correcta desde la carpeta 'notebooks'
ruta_final = './data/processed/inferencia_df_transformado.csv'

inferencia_df.to_csv(ruta_final, index=False)

# --- RESUMEN FINAL PARA CONSOLA ---
print("\n" + "="*30)
print("RESUMEN FINAL:")
print(f"Total de registros guardados: {len(inferencia_df)}")
print(f"Total de columnas: {len(inferencia_df.columns)}")
print(f"Productos únicos: {inferencia_df['producto_id'].nunique()}")
print(f"Rango de fechas: {inferencia_df['fecha'].min()} a {inferencia_df['fecha'].max()}")
print(f"Archivo guardado exitosamente en: {ruta_final}")
print("="*30)

Filtrando solo noviembre y guardando el DataFrame transformado...
Registros totales antes del filtrado: 720
Registros de octubre: 0
Registros de noviembre: 720

⚠️ Atención: Hay 5064 valores nulos en variables de lag en noviembre:
unidades_vendidas_lag1    696
unidades_vendidas_lag2    672
unidades_vendidas_lag3    648
unidades_vendidas_lag4    624
unidades_vendidas_lag5    600
unidades_vendidas_lag6    576
unidades_vendidas_lag7    552
unidades_vendidas_ma7     696
dtype: int64

RESUMEN FINAL:
Total de registros guardados: 720
Total de columnas: 22
Productos únicos: 24
Rango de fechas: 2025-11-01 00:00:00 a 2025-11-30 00:00:00
Archivo guardado exitosamente en: ./data/processed/inferencia_df_transformado.csv


In [21]:
inferencia_df.shape

(720, 71)

In [ ]:
99999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999

In [22]:
inferencia_df.columns

Index(['fecha', 'producto_id', 'nombre', 'categoria', 'subcategoria',
       'precio_base', 'es_estrella', 'unidades_vendidas', 'precio_venta',
       'ingresos', 'año', 'mes', 'dia_mes', 'dia_semana', 'es_fin_semana',
       'es_festivo', 'unidades_vendidas_lag1', 'unidades_vendidas_lag2',
       'unidades_vendidas_lag3', 'unidades_vendidas_lag4',
       'unidades_vendidas_lag5', 'unidades_vendidas_lag6',
       'unidades_vendidas_lag7', 'unidades_vendidas_ma7',
       'descuento_porcentaje', 'precio_competencia', 'ratio_precio',
       'nombre_h_Adidas Own The Run Jacket', 'nombre_h_Adidas Ultraboost 23',
       'nombre_h_Asics Gel Nimbus 25', 'nombre_h_Bowflex SelectTech 552',
       'nombre_h_Columbia Silver Ridge',
       'nombre_h_Decathlon Bandas Elásticas Set', 'nombre_h_Domyos BM900',
       'nombre_h_Domyos Kit Mancuernas 20kg',
       'nombre_h_Gaiam Premium Yoga Block', 'nombre_h_Liforme Yoga Pad',
       'nombre_h_Lotuscrafts Yoga Bolster', 'nombre_h_Manduka PRO Yoga Mat',